In [1]:
import pandas as pd
import numpy as np
import random 
import statistics 
import timeit
from math import factorial

import matplotlib.pyplot as plt

In [5]:
def simulation_Erlang_C(N, servicerate, arrival_rate, termination):
    # Setting paramters
    TNOW = 0
    AreaQ = 0
    AreaP_ = 0
    # Total numbef of people in a system for any given time 
    TotalP= 0
    Total = 0
    AreaP = 0
    Total = 0
    n_served = 0
    N_served =0
    # Average queue 
    Queue = 0 
    free_servers = []
    # Abandonment rate
    Q_num = 0
    # PLotting Parameters for experiment 
    Waiting_number_Served = 0
    Waiting_Time_Served = 0
    
    # Waiting time performance measure 
    Arrival_times = {}
    
    # Setting up the event Calendar 
    col_name = ["Arrivals", "Termination"]

    for i in range(0, N): 
        col_name += ["Server_"+str(i)]
        free_servers += ["Server_"+str(i)]    
        
    # Initializing the event calendar 
    event_calendar = { c : termination+1 for c in col_name}
    event_calendar["Arrivals"] = random.expovariate(arrival_rate)
    event_calendar["Termination"] = termination

 
    # Initialize Record keeping of utilization rates 
    Total_Utilization =  { s : 0 for s in free_servers}
    
    while TNOW < termination:     
        
        TLast = TNOW
        Next_Event = min(event_calendar.keys(), key = (lambda k: event_calendar[k]))
        TNOW = event_calendar[Next_Event]
        
        
        if TNOW >= termination: 
            AreaQ += Queue*(termination - TLast)
            AreaP_ += (TotalP)*(termination - TLast)
            if TotalP >  0:
                AreaP += (Queue/TotalP)*(termination - TLast)

        else:
            AreaQ += Queue*(TNOW - TLast)
            AreaP_ += (TotalP)*(TNOW - TLast)
            if TotalP >  0:
                AreaP += (Queue/TotalP)*(TNOW - TLast)
                
                
        if TNOW < termination:
                  
            if Next_Event == 'Arrivals':  
                event_calendar["Arrivals"] += random.expovariate(arrival_rate)
                # Assign a queue to check if servers are free
                TotalP += 1
                Total +=1
                if free_servers == []:
                    Queue += 1
                    Q_num += 1 
                    Arrival_times[str(Q_num)] = TNOW
                else:
                    n_served +=1
                    s = random.choice(free_servers)
                    # Schedule Service completion and make the server busy
                    free_servers.remove(s)
                    event_calendar[s] = TNOW + random.expovariate(servicerate[s])
                    # Only schedule the next service if TNOW < termination
                    
                    if event_calendar[s] < termination:
                        Total_Utilization[s] += event_calendar[s] - TNOW      
                    else:
                        Total_Utilization[s] += termination - TNOW 
              
            elif (Next_Event).startswith("Server") == True:
                ## Reset the value for the corresponding agent that is now free 
                TotalP -= 1
                N_served += 1
                event_calendar[Next_Event] = termination + 1 
                free_servers.append(Next_Event)
                 # Schedule the next service time once a server is available 
                if Queue > 0 and free_servers != []:
                    s = random.choice(free_servers)
                    Queue -= 1 
                    free_servers.remove(s)
                    event_calendar[s] = TNOW + random.expovariate(servicerate[s])
                    n_served +=1
                    Waiting_Time_Served += TNOW - Arrival_times[list(Arrival_times)[0]]
                    if event_calendar[s] < termination:
                        Total_Utilization[s] += event_calendar[s] - TNOW   
                    else:
                        Total_Utilization[s] += termination - TNOW  
                    Waiting_number_Served += 1 
                    del Arrival_times[list(Arrival_times)[0]]
     
    val = 0
    for k in Total_Utilization:
        val +=  Total_Utilization[k]
    avg_u = val/len(Total_Utilization)/termination
        
    avg_servicerate = 0 
    for k in servicerate:
        avg_servicerate+=servicerate[k]
        
        
    total_servicerate = 0 
    for k in servicerate:
        total_servicerate+= (Total_Utilization[k]/termination)*servicerate[k]
    Abandon = 0

    return Abandon, Q_num, Waiting_Time_Served/N_served,N_served, Waiting_number_Served, avg_u, avg_servicerate, total_servicerate, Total, n_served
